In [ ]:
VER = 6

import pandas as pd, numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()

import os, sys, pickle, glob, gc
from collections import Counter
import cudf, itertools
print('We will use RAPIDS version',cudf.__version__)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

from pandarallel import pandarallel

pandarallel.initialize(nb_workers=4, progress_bar=True, use_memory_fs=False)

# Generating Splits

In [ ]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

In [ ]:
def load_test():    
    dfs = []
    for e, chunk_file in enumerate(glob.glob('../raw_data/test_parquet/*')):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

test_df = load_test()
test_df.to_parquet("./splitted_raw_data/test.parquet")

In [ ]:
def load_train():    
    dfs = []
    for e, chunk_file in enumerate(glob.glob('../raw_data/train_parquet/*')):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

all_train_df = load_train()
all_train_df.to_parquet("./splitted_raw_data/all_train.parquet")

In [ ]:
train_df = pd.read_parquet("../raw_data/radek/train.parquet")
train_df.ts = train_df.ts.astype('int32')
train_df.to_parquet("./splitted_raw_data/train.parquet")

val_df = pd.read_parquet("../raw_data/radek/test.parquet")
val_df.ts = val_df.ts.astype('int32')

np.random.seed(1337)
val_sess = np.random.choice(val_df.session.unique(), val_df.session.nunique()//2)

val_df.to_parquet("./splitted_raw_data/val.parquet")

In [ ]:
val_sess.dump("./splitted_raw_data/val_sessions_for_train.npy")

In [ ]:
val_labels_df = pd.read_parquet("../raw_data/radek/test_labels.parquet")
val_labels_df.to_parquet("./splitted_raw_data/val_labels.parquet")